<a href="https://colab.research.google.com/github/Mohann23/PySpark_Material_Practice/blob/master/Task_6_By_Praveen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

!apt-get install openjdk-8-jdk-headless -qq > /dev/null


!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz



!tar xf spark-3.2.1-bin-hadoop3.2.tgz

!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

import findspark
findspark.init()

findspark.find()
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

Mounted at /content/drive


In [2]:
spark = SparkSession.builder.appName('Task_6').getOrCreate()

In [3]:
user_log_df = spark.read.csv('/content/drive/MyDrive/PYSpark_Files/user_login_history_Task_6.csv',inferSchema=True,header=True)

In [4]:
user_log_df.show(5)

+---+------+----------+
| id|userid|login_date|
+---+------+----------+
|  1|     1|01-01-2019|
|  2|     2|02-01-2019|
|  3|     3|03-01-2019|
|  4|     4|04-01-2019|
|  5|     5|05-01-2019|
+---+------+----------+
only showing top 5 rows



In [5]:
user_log_df = user_log_df.dropna()

Current Year

In [20]:
#Current_year
from datetime import datetime

def try_parsing_date(text):
    for fmt in ('%Y-%m-%d', '%d.%m.%Y', '%d/%m/%Y','%d-%m-%Y','%dd-%mm-%yyyy'):
        try:
            date_object = datetime.strptime(text, fmt)
            converted_date_object = date_object.strftime('%Y')
            return converted_date_object

        except ValueError:
            pass
    return 'ERROR_DATE'


In [14]:
date_udf = udf(lambda x : try_parsing_date(x))

In [15]:
year_convert = user_log_df.withColumn('current_year',date_udf(col('login_date')))

In [16]:
year_convert.show()

+---+------+----------+------------+
| id|userid|login_date|current_year|
+---+------+----------+------------+
|  1|     1|01-01-2019|        2019|
|  2|     2|02-01-2019|        2019|
|  3|     3|03-01-2019|        2019|
|  4|     4|04-01-2019|        2019|
|  5|     5|05-01-2019|        2019|
|  6|     6|06-01-2019|        2019|
|  7|     7|07-01-2019|        2019|
|  8|     8|08-01-2019|        2019|
|  9|    68|08-01-2019|        2019|
| 10|     9|09-01-2019|        2019|
| 11|    10|10-01-2019|        2019|
| 12|     1|01-02-2019|        2019|
| 13|     2|02-02-2019|        2019|
| 14|     3|03-02-2019|        2019|
| 15|     4|04-02-2019|        2019|
| 16|     5|05-02-2019|        2019|
| 17|     6|06-02-2019|        2019|
| 18|     7|07-02-2019|        2019|
| 19|     8|08-02-2019|        2019|
| 20|     9|09-02-2019|        2019|
+---+------+----------+------------+
only showing top 20 rows



In [18]:
current_year = year_convert.groupBy('userid').agg(count(col('current_year')))

In [19]:
current_year.show()

+------+-------------------+
|userid|count(current_year)|
+------+-------------------+
|    31|                 14|
|    34|                 14|
|    28|                 14|
|    26|                 14|
|    27|                 14|
|    44|                 14|
|    12|                 18|
|    22|                 14|
|    47|                 14|
|     1|                 21|
|    13|                 18|
|     6|                 21|
|    16|                 18|
|     3|                 21|
|    20|                 14|
|    40|                 14|
|    48|                 14|
|     5|                 21|
|    19|                 17|
|    41|                 14|
+------+-------------------+
only showing top 20 rows



Previous Year

In [46]:
#Previous Year
from datetime import datetime
from dateutil.relativedelta import relativedelta

def try_parsing_date_py(text):
    for fmt in ('%Y-%m-%d', '%d.%m.%Y', '%d/%m/%Y','%d-%m-%Y','%dd-%mm-%yyyy'):
        try:
            date_object = datetime.strptime(text, fmt)
            last_year = (date_object -relativedelta(years=1))
            converted_date_object = last_year.strftime('%Y')
            
            return last_year

        except ValueError:
            pass
    return 'ERROR_DATE'

In [47]:
date_udf = udf(lambda x : try_parsing_date_py(x))

In [48]:
previous_year_convert = user_log_df.withColumn('Previous_year',date_udf(col('login_date')))

In [50]:
previous_year = previous_year_convert.groupBy('userid').agg(count(col('Previous_year')))

In [51]:
previous_year.show()

+------+--------------------+
|userid|count(Previous_year)|
+------+--------------------+
|    31|                  14|
|    34|                  14|
|    28|                  14|
|    26|                  14|
|    27|                  14|
|    44|                  14|
|    12|                  18|
|    22|                  14|
|    47|                  14|
|     1|                  21|
|    13|                  18|
|     6|                  21|
|    16|                  18|
|     3|                  21|
|    20|                  14|
|    40|                  14|
|    48|                  14|
|     5|                  21|
|    19|                  17|
|    41|                  14|
+------+--------------------+
only showing top 20 rows



In [39]:
try_parsing_date('08-01-2019')

datetime.datetime(2018, 1, 8, 0, 0)

Current month

In [52]:
from datetime import datetime

def try_parsing_date(text):
    for fmt in ('%Y-%m-%d', '%d.%m.%Y', '%d/%m/%Y','%d-%m-%Y','%dd-%mm-%yyyy'):
        try:
            date_object = datetime.strptime(text, fmt)
            converted_date_object = date_object.strftime('%m')
            return converted_date_object

        except ValueError:
            pass
    return 'ERROR_DATE'

In [53]:
date_udf = udf(lambda x : try_parsing_date(x))

In [60]:
current_month= user_log_df.withColumn('current_month',date_udf(col('login_date')))

In [61]:
current_month.show()

+---+------+----------+-------------+
| id|userid|login_date|current_month|
+---+------+----------+-------------+
|  1|     1|01-01-2019|           01|
|  2|     2|02-01-2019|           01|
|  3|     3|03-01-2019|           01|
|  4|     4|04-01-2019|           01|
|  5|     5|05-01-2019|           01|
|  6|     6|06-01-2019|           01|
|  7|     7|07-01-2019|           01|
|  8|     8|08-01-2019|           01|
|  9|    68|08-01-2019|           01|
| 10|     9|09-01-2019|           01|
| 11|    10|10-01-2019|           01|
| 12|     1|01-02-2019|           02|
| 13|     2|02-02-2019|           02|
| 14|     3|03-02-2019|           02|
| 15|     4|04-02-2019|           02|
| 16|     5|05-02-2019|           02|
| 17|     6|06-02-2019|           02|
| 18|     7|07-02-2019|           02|
| 19|     8|08-02-2019|           02|
| 20|     9|09-02-2019|           02|
+---+------+----------+-------------+
only showing top 20 rows



In [63]:
current_Month = current_month.groupBy('userid').agg(count(col('Current_month')))

In [64]:
current_Month.show()

+------+--------------------+
|userid|count(Current_month)|
+------+--------------------+
|    31|                  14|
|    34|                  14|
|    28|                  14|
|    26|                  14|
|    27|                  14|
|    44|                  14|
|    12|                  18|
|    22|                  14|
|    47|                  14|
|     1|                  21|
|    13|                  18|
|     6|                  21|
|    16|                  18|
|     3|                  21|
|    20|                  14|
|    40|                  14|
|    48|                  14|
|     5|                  21|
|    19|                  17|
|    41|                  14|
+------+--------------------+
only showing top 20 rows



Previous month

In [71]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

def try_parsing_date_cm(text):
    for fmt in ('%Y-%m-%d', '%d.%m.%Y', '%d/%m/%Y','%d-%m-%Y','%dd-%mm-%yyyy'):
        try:
            date_object = datetime.strptime(text, fmt)
            last_month = date_object - relativedelta(months=1)
            converted_date_object = last_month.strftime('%m')
            return converted_date_object

        except ValueError:
            pass
    return 'ERROR_DATE'

In [73]:
date_udf = udf(lambda x : try_parsing_date_cm(x))

In [74]:
previous_month= user_log_df.withColumn('Previous_month',date_udf(col('login_date')))

In [75]:
previous_month.show()

+---+------+----------+--------------+
| id|userid|login_date|Previous_month|
+---+------+----------+--------------+
|  1|     1|01-01-2019|            12|
|  2|     2|02-01-2019|            12|
|  3|     3|03-01-2019|            12|
|  4|     4|04-01-2019|            12|
|  5|     5|05-01-2019|            12|
|  6|     6|06-01-2019|            12|
|  7|     7|07-01-2019|            12|
|  8|     8|08-01-2019|            12|
|  9|    68|08-01-2019|            12|
| 10|     9|09-01-2019|            12|
| 11|    10|10-01-2019|            12|
| 12|     1|01-02-2019|            01|
| 13|     2|02-02-2019|            01|
| 14|     3|03-02-2019|            01|
| 15|     4|04-02-2019|            01|
| 16|     5|05-02-2019|            01|
| 17|     6|06-02-2019|            01|
| 18|     7|07-02-2019|            01|
| 19|     8|08-02-2019|            01|
| 20|     9|09-02-2019|            01|
+---+------+----------+--------------+
only showing top 20 rows



In [76]:
previous_Month = previous_month.groupBy('userid').agg(count(col('Previous_month')))

In [77]:
previous_Month.show()

+------+---------------------+
|userid|count(Previous_month)|
+------+---------------------+
|    31|                   14|
|    34|                   14|
|    28|                   14|
|    26|                   14|
|    27|                   14|
|    44|                   14|
|    12|                   18|
|    22|                   14|
|    47|                   14|
|     1|                   21|
|    13|                   18|
|     6|                   21|
|    16|                   18|
|     3|                   21|
|    20|                   14|
|    40|                   14|
|    48|                   14|
|     5|                   21|
|    19|                   17|
|    41|                   14|
+------+---------------------+
only showing top 20 rows



Current week

In [78]:

from datetime import *

def check_week(str1):
  current_date = datetime.strptime(str1,'%d-%m-%Y')     #checking that the date format
  current_date = current_date.date()    #using this line we are changing the date format without hrs and minutes format
  today_date = date.today()    #Present date 
  week_start_date = today_date - timedelta(days=today_date.weekday())     #Start date of the week
  week_end_date = week_start_date +timedelta(days=6)     #End date of the week
  
  if week_start_date <= current_date <= week_end_date:
    return 't'
  else:
    return 'f'

In [79]:
current_week_udf = udf(lambda x:check_week(x))

In [80]:
current_week= user_log_df.withColumn('current_week',current_week_udf('login_date'))

In [81]:
current_week.show()

+---+------+----------+------------+
| id|userid|login_date|current_week|
+---+------+----------+------------+
|  1|     1|01-01-2019|           f|
|  2|     2|02-01-2019|           f|
|  3|     3|03-01-2019|           f|
|  4|     4|04-01-2019|           f|
|  5|     5|05-01-2019|           f|
|  6|     6|06-01-2019|           f|
|  7|     7|07-01-2019|           f|
|  8|     8|08-01-2019|           f|
|  9|    68|08-01-2019|           f|
| 10|     9|09-01-2019|           f|
| 11|    10|10-01-2019|           f|
| 12|     1|01-02-2019|           f|
| 13|     2|02-02-2019|           f|
| 14|     3|03-02-2019|           f|
| 15|     4|04-02-2019|           f|
| 16|     5|05-02-2019|           f|
| 17|     6|06-02-2019|           f|
| 18|     7|07-02-2019|           f|
| 19|     8|08-02-2019|           f|
| 20|     9|09-02-2019|           f|
+---+------+----------+------------+
only showing top 20 rows



In [83]:
current_week_after_grouping = current_week.filter(col('current_week')=='f').groupBy('userid').agg(count(col('current_week')))

In [84]:
current_week_after_grouping.show()

+------+-------------------+
|userid|count(current_week)|
+------+-------------------+
|    31|                 14|
|    34|                 14|
|    28|                 14|
|    26|                 14|
|    27|                 14|
|    44|                 14|
|    12|                 18|
|    22|                 14|
|    47|                 14|
|     1|                 21|
|    13|                 18|
|     6|                 21|
|    16|                 17|
|     3|                 21|
|    20|                 14|
|    40|                 14|
|    48|                 14|
|     5|                 21|
|    19|                 17|
|    41|                 14|
+------+-------------------+
only showing top 20 rows



Previous year

In [85]:
from datetime import *
def check_week_previous(str1):
  current_date = datetime.strptime(str1,'%d-%m-%Y')     #checking that the date format
  current_date = current_date.date()    #using this line we are changing the date format without hrs and minutes format
  today_date = date.today()    #Present date 
  week_start_date = today_date - timedelta(days=today_date.weekday())     #Start date of the week
  week_end_date = week_start_date +timedelta(days=6)     #End date of the week
  previous_week_start = week_start_date - timedelta(days=7)  #Start of previous week
  previous_week_end = previous_week_start + timedelta(days=6)    #end of previous week
  if previous_week_start <= week_start_date <=previous_week_end:
    return 't'
  else:
    return 'f'

In [86]:
previous_week_udf = udf(lambda x:check_week_previous(x))

In [87]:
previous_week = user_log_df.withColumn('Previous_week',current_week_udf('login_date'))

In [88]:
previous_week.show()

+---+------+----------+-------------+
| id|userid|login_date|Previous_week|
+---+------+----------+-------------+
|  1|     1|01-01-2019|            f|
|  2|     2|02-01-2019|            f|
|  3|     3|03-01-2019|            f|
|  4|     4|04-01-2019|            f|
|  5|     5|05-01-2019|            f|
|  6|     6|06-01-2019|            f|
|  7|     7|07-01-2019|            f|
|  8|     8|08-01-2019|            f|
|  9|    68|08-01-2019|            f|
| 10|     9|09-01-2019|            f|
| 11|    10|10-01-2019|            f|
| 12|     1|01-02-2019|            f|
| 13|     2|02-02-2019|            f|
| 14|     3|03-02-2019|            f|
| 15|     4|04-02-2019|            f|
| 16|     5|05-02-2019|            f|
| 17|     6|06-02-2019|            f|
| 18|     7|07-02-2019|            f|
| 19|     8|08-02-2019|            f|
| 20|     9|09-02-2019|            f|
+---+------+----------+-------------+
only showing top 20 rows



In [89]:
previous_week_after_grouping = previous_week.filter(col('Previous_week')=='f').groupBy('userid').agg(count(col('Previous_week'))) 

In [90]:
previous_week_after_grouping.show()

+------+--------------------+
|userid|count(Previous_week)|
+------+--------------------+
|    31|                  14|
|    34|                  14|
|    28|                  14|
|    26|                  14|
|    27|                  14|
|    44|                  14|
|    12|                  18|
|    22|                  14|
|    47|                  14|
|     1|                  21|
|    13|                  18|
|     6|                  21|
|    16|                  17|
|     3|                  21|
|    20|                  14|
|    40|                  14|
|    48|                  14|
|     5|                  21|
|    19|                  17|
|    41|                  14|
+------+--------------------+
only showing top 20 rows

